# Notebook pro trénink s destilací nad datasetem CIFAR10
V tomto notebooku je trénován MobileNetV2 nad datasetem CIFAR10, jako učitelsý model je využíván finetunued ViT nad stejným datasetem. 

MobileNetV2 je používán s náhodnou inicializací, tréninkem pouze klasifikační hlavy inicializovaného (předtrénovaného nad ImageNetem) MobileNetuV2 a trénink celého modelu, taktéž inicializovaného. Tyto tři úlohy jsou trénovány bězným způsobem a také s pomocí destilace výše zmíněného modelu.  

Při destilaci je využíváno předpočítaných logitů ze sešitu precompute_logits.

## Import knihoven a definice metod

In [2]:
from transformers import Trainer
from torch.utils.data import ConcatDataset
import pandas as pd
import optuna
import torch
import math
import base
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [3]:
dataset_part = base.get_dataset_part()

Resetování náhodného seedu pro replikovatelnost výsledků.

In [4]:
base.reset_seed()

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A40


Provedení transformací nad datasetem.

In [6]:
DATASET = "cifar10"

In [7]:
transform = base.base_transforms()

#Poslední train batch použijeme jako eval část...
test = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TEST, transform=transform)
train = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TRAIN, transform=transform)
eval = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.EVAL, transform=transform)

In [8]:
augment_transform = base.aug_transforms()
train_aug = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TRAIN, transform=augment_transform)
train_aug = base.remove_diff_pred_class(train, train_aug, pytorch_dataset=True)

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

In [9]:
train_combo = ConcatDataset([train, train_aug])

In [10]:
# Test rozložení --> Good Enough
df = pd.DataFrame(eval.labels)
print(df.value_counts())

0
5    1025
9    1022
3    1016
0    1014
1    1014
8    1003
4     997
6     980
7     977
2     952
Name: count, dtype: int64


### Standardní trénink náhodně inicializovaného modelu. 

In [11]:
num_epochs = 10
batch_size = 128

In [12]:
#Nápočet epoch na steps
data_length = len(train)
min_r = math.ceil(data_length/batch_size)*3
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [13]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

In [14]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [15]:
base.reset_seed()

In [16]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/random_hp-search", logging_dir=f"~/logs/{DATASET}/random_hp-search", epochs=num_epochs, batch_size=batch_size)

In [17]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.get_random_init_mobilenet(10)
)
  

In [18]:
best_base_random = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-random",
    n_trials=150
)

[I 2025-04-07 19:01:45,083] A new study created in memory with name: Base-random


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.542700,1.065142,0.620800,0.623963,0.620284,0.613948
2,0.962800,0.759808,0.733100,0.738952,0.732427,0.732924


[I 2025-04-07 19:04:12,798] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.494600,1.129562,0.611300,0.620089,0.611040,0.606143
2,0.977000,0.761083,0.732700,0.739908,0.731687,0.730955
3,0.759700,0.651948,0.773200,0.775197,0.773640,0.770788
4,0.607300,0.533318,0.817200,0.816079,0.816900,0.814824
5,0.486800,0.513817,0.823200,0.827947,0.823414,0.822361
6,0.381100,0.472339,0.843000,0.846422,0.843006,0.843700
7,0.281400,0.451907,0.852600,0.852351,0.852906,0.851967
8,0.189200,0.453905,0.857800,0.857712,0.857869,0.857432
9,0.107500,0.508827,0.852700,0.858736,0.852542,0.854167
10,0.055800,0.518586,0.855100,0.857181,0.855476,0.854648


[I 2025-04-08 11:51:57,816] Trial 148 finished with value: 0.8546481014940118 and parameters: {'learning_rate': 0.0005005350996293366, 'weight_decay': 0.007, 'warmup_steps': 15}. Best is trial 136 with value: 0.8613704644687414.


Trial 149 with params: {'learning_rate': 0.00033278420397738303, 'weight_decay': 0.007, 'warmup_steps': 18}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.505100,1.068179,0.621500,0.640184,0.621841,0.615639
2,0.937300,0.747414,0.740500,0.750662,0.739552,0.741991
3,0.708500,0.633167,0.782100,0.784100,0.782782,0.778653
4,0.557100,0.532129,0.817200,0.816286,0.817077,0.815252
5,0.430800,0.501865,0.831000,0.833005,0.831142,0.829452
6,0.320000,0.491733,0.836700,0.842868,0.836959,0.837897
7,0.215600,0.509141,0.840200,0.844179,0.840719,0.840130
8,0.126000,0.505209,0.847600,0.849351,0.847511,0.847995


[I 2025-04-08 12:01:39,594] Trial 149 pruned. 


In [19]:
print(best_base_random)

BestRun(run_id='136', objective=0.8613704644687414, hyperparameters={'learning_rate': 0.0005280489047611394, 'weight_decay': 0.008, 'warmup_steps': 13}, run_summary=None)


In [15]:
base.reset_seed()

In [16]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/random-KD_hp-search", logging_dir=f"~/logs/{DATASET}/random-KD_hp-search",  remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [17]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [18]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [19]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.get_random_init_mobilenet(10)
)
  

In [20]:
best_distill_random = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-04-04 19:42:45,231] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.088800,0.807692,0.620300,0.628576,0.619397,0.615550
2,0.708000,0.561188,0.750100,0.757068,0.749581,0.750302


[I 2025-04-04 19:46:46,660] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 28, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.047400,0.786290,0.627900,0.624710,0.627158,0.620713
2,0.731600,0.576808,0.735900,0.754070,0.735151,0.738266


[I 2025-04-06 00:44:37,904] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0007948125851896977, 'weight_decay': 0.0, 'warmup_steps': 17, 'lambda_param': 0.0, 'temperature': 7.0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.048100,0.843022,0.601400,0.608558,0.601168,0.595615
2,0.738600,0.589394,0.736300,0.742092,0.735589,0.736877
3,0.597900,0.509572,0.784800,0.789235,0.785407,0.781890
4,0.509000,0.440768,0.814300,0.813443,0.813995,0.811257
5,0.440200,0.404485,0.832500,0.839022,0.832304,0.832647
6,0.383900,0.385286,0.847100,0.850472,0.847083,0.847701
7,0.332900,0.351667,0.860000,0.862348,0.860270,0.860257
8,0.287700,0.344766,0.863800,0.863883,0.863938,0.863393


[I 2025-04-06 01:00:31,309] Trial 149 pruned. 


In [21]:
print(best_distill_random)

BestRun(run_id='61', objective=0.8750916744350314, hyperparameters={'learning_rate': 0.0005943410799444305, 'weight_decay': 0.001, 'warmup_steps': 30, 'lambda_param': 0.30000000000000004, 'temperature': 6.0}, run_summary=None)


In [22]:
base.reset_seed()

In [23]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained-head_hp-search", logging_dir=f"~/logs/{DATASET}/pretrained-head_hp-search", epochs=num_epochs, batch_size=batch_size)

In [24]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

In [25]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [26]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.freeze_model(base.get_mobilenet(10))
)
  

config.json:   0%|          | 0.00/69.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

In [27]:
best_base_head = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-head",
    n_trials=150
)

[I 2025-04-06 01:09:59,492] A new study created in memory with name: Base-head


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.302000,0.833241,0.733600,0.733584,0.732961,0.732005
2,0.957700,0.751208,0.751400,0.753175,0.751159,0.750540


[I 2025-04-06 01:12:47,512] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.026600,0.715247,0.751700,0.755686,0.751605,0.750060
2,0.899700,0.692088,0.764200,0.769434,0.764082,0.764753
3,0.879100,0.679958,0.766100,0.770169,0.765898,0.763862
4,0.860900,0.661131,0.774700,0.776709,0.774311,0.773267
5,0.852800,0.666512,0.773500,0.774870,0.773127,0.771100
6,0.845300,0.688659,0.763700,0.778013,0.763366,0.766413
7,0.836900,0.664345,0.772700,0.775636,0.772908,0.772257
8,0.831400,0.655118,0.776900,0.778330,0.776659,0.776396


[I 2025-04-06 22:07:53,019] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0007248597626666188, 'weight_decay': 0.002, 'warmup_steps': 30}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.117600,0.749488,0.746200,0.749249,0.745771,0.744800
2,0.893500,0.700712,0.763000,0.765795,0.762845,0.761790


[I 2025-04-06 22:10:44,090] Trial 149 pruned. 


In [28]:
print(best_base_head)

BestRun(run_id='71', objective=0.7745430633961783, hyperparameters={'learning_rate': 0.001886198183438793, 'weight_decay': 0.004, 'warmup_steps': 26}, run_summary=None)


In [29]:
base.reset_seed()

In [30]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained-head-KD_hp-search", logging_dir=f"~/logs/{DATASET}/pretrained-head-KD_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [31]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [32]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [33]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.freeze_model(base.get_mobilenet(10))
)

In [34]:
best_distill_head = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-04-06 23:16:45,934] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.917100,0.685989,0.736300,0.738420,0.735711,0.735201
2,0.766900,0.652735,0.758400,0.759606,0.758088,0.757283


[I 2025-04-06 23:19:35,853] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 28, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.832800,0.655299,0.753800,0.759043,0.753415,0.753484
2,0.753900,0.640535,0.767800,0.771372,0.767430,0.767530
3,0.743200,0.632483,0.765500,0.769094,0.765174,0.763815
4,0.738100,0.619975,0.772000,0.775938,0.771284,0.771198
5,0.734800,0.619339,0.771500,0.773393,0.771082,0.769855
6,0.731400,0.629653,0.759500,0.769368,0.759120,0.761165
7,0.730700,0.629712,0.766200,0.771660,0.766330,0.766046
8,0.728000,0.621453,0.768600,0.768619,0.768159,0.767778


[I 2025-04-07 18:36:07,274] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.00069247511617349, 'weight_decay': 0.01, 'warmup_steps': 10, 'lambda_param': 0.4, 'temperature': 3.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.837500,0.656667,0.753700,0.759171,0.753298,0.753431
2,0.753800,0.641344,0.765500,0.769500,0.765109,0.765373
3,0.742900,0.632783,0.765000,0.767720,0.764698,0.763170
4,0.737700,0.619541,0.772100,0.776025,0.771385,0.771273
5,0.734300,0.619081,0.771800,0.773689,0.771365,0.770208
6,0.731000,0.629887,0.759200,0.768921,0.758825,0.760848
7,0.730400,0.629897,0.766200,0.771825,0.766327,0.766089
8,0.727700,0.621657,0.768800,0.768862,0.768354,0.767995


[I 2025-04-07 18:47:20,929] Trial 149 pruned. 


In [35]:
print(best_distill_head)

BestRun(run_id='68', objective=0.771047008351801, hyperparameters={'learning_rate': 0.0008350401275702886, 'weight_decay': 0.008, 'warmup_steps': 1, 'lambda_param': 0.30000000000000004, 'temperature': 4.0}, run_summary=None)


In [14]:
base.reset_seed()

In [15]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained_hp-search", logging_dir=f"~/logs/{DATASET}/pretrained_hp-search", epochs=num_epochs, batch_size=batch_size)

In [16]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

In [17]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [18]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.get_mobilenet(10)
)
  

In [ ]:
best_base_pretrained = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-04-01 22:33:40,519] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.459600,0.258939,0.911600,0.915043,0.911785,0.911368
2,0.159800,0.193215,0.937300,0.937890,0.937633,0.937199


[I 2025-04-01 22:37:42,262] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.418800,0.243009,0.913500,0.917394,0.913725,0.913686
2,0.155700,0.195179,0.936200,0.936412,0.936463,0.936074
3,0.077000,0.214537,0.935900,0.935876,0.936400,0.935453
4,0.044300,0.255471,0.934900,0.936557,0.934980,0.935083


[I 2025-04-03 02:43:04,278] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0004159019964208297, 'weight_decay': 0.003, 'warmup_steps': 18}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.449000,0.272625,0.910300,0.913168,0.910310,0.910191
2,0.184200,0.217769,0.928400,0.930273,0.928638,0.928418
3,0.103000,0.214900,0.930900,0.930603,0.931259,0.930556
4,0.064800,0.210533,0.940500,0.940802,0.940574,0.940344
5,0.036000,0.239283,0.940700,0.940915,0.940800,0.940567
6,0.024300,0.260252,0.939800,0.940612,0.940106,0.939974
7,0.012700,0.260881,0.944100,0.944476,0.944323,0.944119
8,0.004500,0.252123,0.948700,0.949794,0.948752,0.949044
9,0.001900,0.259649,0.948400,0.949849,0.948443,0.948842
10,0.000700,0.239273,0.950800,0.950834,0.951100,0.950712


[I 2025-04-03 03:02:58,143] Trial 149 finished with value: 0.9507122835349253 and parameters: {'learning_rate': 0.0004159019964208297, 'weight_decay': 0.003, 'warmup_steps': 18}. Best is trial 8 with value: 0.9521566790618318.


In [ ]:
print(best_base_pretrained)

BestRun(run_id='8', objective=0.9521566790618318, hyperparameters={'learning_rate': 0.00040842279473800845, 'weight_decay': 0.008, 'warmup_steps': 6}, run_summary=None)


In [ ]:
base.reset_seed()

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained-KD_hp-search", logging_dir=f"~/logs/{DATASET}/pretrained-KD_hp-search",  remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [ ]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [ ]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [ ]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.get_mobilenet(10)
)

In [ ]:
best_distil_pretrained = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-04-03 03:03:15,965] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.402900,0.257398,0.917600,0.921376,0.917812,0.917882
2,0.225900,0.202859,0.941900,0.942554,0.942269,0.941777


[I 2025-04-03 03:07:18,533] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 28, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.571000,0.401291,0.836900,0.844332,0.836938,0.836957
2,0.386200,0.340433,0.864300,0.870963,0.864551,0.863232


[I 2025-04-04 08:10:46,902] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0005596762209877068, 'weight_decay': 0.0, 'warmup_steps': 11, 'lambda_param': 1.0, 'temperature': 4.0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.389000,0.283152,0.904200,0.909652,0.904084,0.904652
2,0.246300,0.228468,0.929200,0.930023,0.929583,0.928733
3,0.200400,0.225721,0.931700,0.931462,0.932165,0.931057
4,0.171900,0.207539,0.942500,0.943298,0.942479,0.942537
5,0.155000,0.191063,0.947800,0.948282,0.948104,0.947851
6,0.145100,0.189687,0.947500,0.948251,0.947817,0.947658
7,0.140600,0.184649,0.950200,0.950633,0.950586,0.950218
8,0.136500,0.178124,0.954100,0.954708,0.954343,0.954301
9,0.134200,0.177650,0.952400,0.953967,0.952533,0.952840
10,0.132700,0.176178,0.953400,0.953563,0.953736,0.953345


[I 2025-04-04 08:30:46,772] Trial 149 finished with value: 0.9533447571219416 and parameters: {'learning_rate': 0.0005596762209877068, 'weight_decay': 0.0, 'warmup_steps': 11, 'lambda_param': 1.0, 'temperature': 4.0}. Best is trial 23 with value: 0.957889508474951.


In [ ]:
print(best_distil_pretrained)

BestRun(run_id='23', objective=0.957889508474951, hyperparameters={'learning_rate': 0.0005261925323189436, 'weight_decay': 0.006, 'warmup_steps': 24, 'lambda_param': 0.6000000000000001, 'temperature': 6.0}, run_summary=None)


In [ ]:
print("Best random init training score: ", best_base_random)
print("Best random init distilation trianing score: ", best_distill_random)
print("Best pretrained (head only) training score: ", best_base_head)
print("Best pretrained distilation (head only) training score: ",best_distill_head)
print("Best pretrained training score: ", best_base_pretrained)
print("Best pretrained distilation training score: ", best_distil_pretrained)

NameError: name 'best_base_random' is not defined